## 代码功能
- 计算出完全免考核的的充放电动作量
- 考虑到储能的各种参数
- 计算速度快，有利于网格搜索


In [1]:
import pandas as pd
import numpy as np
PReal = pd.DataFrame(pd.read_csv('std_data_5min.csv'))["std_realpower"]
# 10min考核点
# 由5min构造10min考核点时，由于是后向平均，需要用i点和i点前一个数据求和除以二
# 105119
PReal = np.array(PReal)  # 确保PReal是一个numpy数组
averages = (PReal[:-1] + PReal[1:]) / 2.0
averages = np.append(averages, averages[-1]) # 保持数组长度一致
import cvxpy as cp
# 运行一年时间的数据（10万5000个数据）需要约62min，大约1小时
# 为averages数据

# 创建一个与averages同样长度的变量数组，每个元素代表对应点的调整量
adjustments = cp.Variable(len(averages))

# 目标是最小化所有调整量的平方和
objective = cp.Minimize(cp.sum_squares(adjustments))

# 约束条件，确保所有调整后的相邻点的变化量小于等于0.022
constraints = [cp.abs(averages[i] + adjustments[i] - averages[i+2] - adjustments[i+2]) <= 0.022
               for i in range(len(averages) - 2)]

# 添加约束，确保调整后的数组首尾元素的变化量也小于等于0.022
constraints += [cp.abs(adjustments[0] - adjustments[-1]) <= 0.022]

# 定义问题并求解
problem = cp.Problem(objective, constraints)
problem.solve()

# 打印结果
print("调整量:", adjustments.value)
print("调整后的数组:", averages + adjustments.value)
Pc_thy = adjustments.value


In [1]:
# 创建一个DataFrame
df = pd.DataFrame({
    'Pc_thy': Pc_thy,
    'averages': averages
})
# 将DataFrame保存为CSV文件
csv_file_path = 'std_Pc_thy.csv'
df.to_csv(csv_file_path, index=False)
pd.DataFrame(Pc_thy).to_csv("Pc_thy.csv")

NameError: name 'pd' is not defined